In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.gof
import statsmodels.api

# Chi-squared categorical

This notebook presents some simple examples of a chi-squared test on categorical data.

## Introduction

Wikipedia's article on the [chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test) gives an overview , referencing the [chi-squared distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution) of $k$ degrees of freedom; $\chi^2_k \sim \sum_{i=1}^k Z^2_i$, where the $Z_i$ are independent, and $Z_i \sim N(0,1)$.

The typical form of a chi-squared test is a [Pearson's chi-squred test](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test). The degrees of freedom depends on the type of test, which may be a test of 
- goodness-of-fit
- homogeneity
- independence

For a population split into categories, the null hypothesis for (Pearson's) chi-squared test is that the there are no differences between the categories.

## Simple example
Consider 2 clothing stores, Primark, Next and Debenhams. They all sell shirts.

Does a particular store significantly sell more shirts than another? The null hypothesis becomes that the number of each store's shirts comes from the same distribution.

### When the categories are all similar
Visiting a canididate stores on the high street, one can count the number of shirts in the stores. This might yield the results

In [2]:
df = pd.DataFrame({"shirts": {"Primark": 28, "Debenhams": 32, "Next": 27}})
print(df)

           shirts
Debenhams      32
Next           27
Primark        28


where the number of blue shirts is always roughly twice the number of red shirts. There are 3 stores, and $28 + 27 + 32 = 87$ shirts in total, yielding an expected number of 29 shirts per store. One can then go ahead and calculate a $\chi^2$ test statistic

$\chi^2 = \sum_i \frac{(O_i - E_i)^2}{E_i}$

which superficially, looks like a sum of variables $\frac{(x-\mu)^2}{\sigma}$. In the limit of the number of test samples going to infinity, the test statistic approaches a true chi-squared distribution. In this case

In [3]:
e_i = np.mean(df["shirts"])
chi_2 = np.sum((df["shirts"].array - e_i) ** 2 / e_i)
print(f"chi squared test stat = {chi_2}")

chi squared test stat = 0.4827586206896552


There are 3 cells, and the calculation is made on differences, so the degrees of freedom is 3 - 1 = 2. This allows calculation of a one-sided p-value.

In [4]:
p_value = 1.0 - scipy.stats.chi2.cdf(chi_2, 2)
print(f"P-value = {p_value}")

P-value = 0.7855436050549208


So the probability of getting a result more extreme than the test statistic is 0.786. At an reasonable significance level, (e.g. 5% or 10%), this would lead to accepting the null hypothesis that the categories are all statistically similar. 

### When the categories are dis-similar

Consider now a heavily imbalanced class case.

In [5]:
df = pd.DataFrame({"shirts": {"Primark": 20, "Debenhams": 21, "Next": 46}})
print(df)
e_i = np.mean(df["shirts"])
chi_2 = np.sum((df["shirts"].array - e_i) ** 2 / e_i)
print(f"chi squared test stat = {chi_2}")
p_value = 1.0 - scipy.stats.chi2.cdf(chi_2, 2)
print(f"P-value = {p_value}")

           shirts
Debenhams      21
Next           46
Primark        20
chi squared test stat = 14.96551724137931
P-value = 0.000562702988414987


Thus the probability of getting a more extreme example is 0.0006. At a reasonable significance level, e.g. 5%, this would lead to a rejection of the null hypothesis, and one would conclude that the categories are different.

### Calculation using libs

Scipy has a chisquare routine, where one can pass in the raw data to obtain the test statistic and p-value. The routine accepts an argument ddof, which affects the $k$ value used by the test. By default, k will be the length of the input array (call it N) minus 1. With a ddof argument, this becomes $k = N - 1 - \textrm{ddof}$.

In [6]:
scipy.stats.chisquare(df["shirts"], ddof=0)

Power_divergenceResult(statistic=14.96551724137931, pvalue=0.0005627029884150075)

These numbers can also be obtained with statsmodels

In [7]:
statsmodels.stats.gof.chisquare(df["shirts"], ddof=0)

(14.96551724137931, 0.0005627029884150075)

## Multiple categories or sample sets - homogeneity/independence

Consider the case where now each of the stores sells multiple apparel, e.g. vests

In [8]:
df = pd.DataFrame({"shirts": {"Primark": 28, "Debenhams": 32, "Next": 27}, "vests": {"Primark": 61, "Debenhams": 62, "Next": 57}})
df_totals = df.copy()
df_totals.loc['Total']= df_totals.sum(numeric_only=True, axis=0)
df_totals.loc[:,'Total'] = df_totals.sum(numeric_only=True, axis=1)
print(df)
print(df_totals)

           shirts  vests
Primark        28     61
Debenhams      32     62
Next           27     57
           shirts  vests  Total
Primark        28     61     89
Debenhams      32     62     94
Next           27     57     84
Total          87    180    267


The totals can be used to get the 'expected' number of elements in each cell

In [9]:
def get_expected(df_tots, row_id, column_id):
    return df_tots.loc[row_id, "Total"] * df_tots.loc["Total", column_id] / df_tots.loc["Total", "Total"]
print(f"Expected number of Primark shirts = {get_expected(df_totals, 'Primark', 'shirts')}")
print(f"Expected number of Primark vests = {get_expected(df_totals, 'Primark', 'vests')}")

Expected number of Primark shirts = 29.0
Expected number of Primark vests = 60.0


This allows a calculation of a chi-squared test statistic

In [10]:
chi_2 = 0.0
for row_id in df.index:
    for col_id in df.columns:
        o_i = df.loc[row_id, col_id]
        e_i = get_expected(df_totals, row_id, col_id)
        chi_2 += (o_i - e_i) ** 2 / e_i
print(f"chi squared test stat = {chi_2}")

chi squared test stat = 0.14960040876218433


For tabular data, since the calculation comes down to differences, the number of degrees of freedom are (number of rows -1) x (number of columns - 1) = (3-1) x (2-1) = 2. This allows calculation of a p-value

In [11]:
p_value = 1.0 - scipy.stats.chi2.cdf(chi_2, 2)
print(f"P-value = {p_value}")

P-value = 0.9279288639307913


Thus the probability of getting a more extreme example is 0.928. So far this is all calculation. Consider, now, the hypothesis test questions that can be posed. The data has 2 parameters, so the variety of questions is larger.

There is a question of 'independence' of store. In this case, the null hypothesis would be that the apparel population is independent of store.

There is also a question (which is related, but not necessarily the same) of 'homogeneity'. In this case, the null hypothesis would be that the distribution of shirts and vests is the same amongst all stores.

In both cases, the calculation is the same. At an reasonable significance level, (e.g. 5% or 10%), in both cases, this would lead to accepting the null hypothesis.

### Calculation using libs

[scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can be used to perform the calculation, returning a 4-tuple of (test stat, p-value, degrees of freedom, expected values)

In [12]:
scipy.stats.chi2_contingency(df)

(0.14960040876218433,
 0.9279288639307913,
 2,
 array([[29.        , 60.        ],
        [30.62921348, 63.37078652],
        [27.37078652, 56.62921348]]))

[Contingency tables](https://www.statsmodels.org/dev/contingency_tables.html) in statsmodels can be useful for seeing the contributions to the test statistic (each element being the respective value for $\frac{(O_i - E_i)^2}{E_i}$). The sum of the contributions retrieves the test statistic (which can be handy for seeing the entry that contribute heavily to breaking homogeneity/independence)

In [13]:
table = statsmodels.api.stats.Table(df)
table.chi2_contribs

,shirts,vests
Primark,0.034483,0.016667
Debenhams,0.061348,0.029652
Next,0.005023,0.002428


In [14]:
chi2 = table.chi2_contribs.to_numpy().sum()
print(f"chi squared test stat = {chi_2}")

chi squared test stat = 0.14960040876218433


## Goodness of fit

Consider the case now where a distributor knows how many shirts should be in the local stores, i.e. there is a predefined distribution. How close is the observation to the distributor's knowledge? The null hypothesis is that the observed distribution is sampled from the the predefined distribution.

In [15]:
df = pd.DataFrame({"shirts": {"Primark": 28, "Debenhams": 32, "Next": 27}, 
                   "expected_shirts": {"Primark": 30, "Debenhams": 35, "Next": 30}})
print(df)

           shirts  expected_shirts
Primark        28               30
Debenhams      32               35
Next           27               30


This can be framed as a chi-squared test, where the null hypothesis is that the observed distribution is a "good fit" to the expected distribution.

To do this, the two distributions need to be normalised (i.e. sum to the same number, so that they can be treated as distributions over the same space).

In [16]:
df["expected_shirts_normed"] = df["expected_shirts"] * df["shirts"].sum() / df["expected_shirts"].sum()
print(df)

           shirts  expected_shirts  expected_shirts_normed
Primark        28               30               27.473684
Debenhams      32               35               32.052632
Next           27               30               27.473684


This is a slightly different calculation from before, since now all the $E_i$ have been supplied. The test statistic now becomes

In [17]:
chi_2 = np.sum((df["shirts"] - df["expected_shirts_normed"]) ** 2 / df["expected_shirts_normed"])
print(f"chi squared test stat = {chi_2}")

chi squared test stat = 0.018336070060208


The degrees of freedom $k$ are given by $k = n - 1 - q$, where $n$ is the number of non-empty cells (3 in this case) and $p$ is the number of parameters in the model (e.g. for a 1D line of best fit, $y=mx+c$, this would have $q=2$). In this case, since no free parameters specify the fit, $q=0$. This means that a p-value is calculable

In [18]:
p_value = 1.0 - scipy.stats.chi2.cdf(chi_2, 3-1)
print(f"P-value = {p_value}")

P-value = 0.9908738632636134


There is a 0.991 probability of an observation being more extreme than the observed observation, which for any reasonable significance level would mean accepting the null hypothesis.

### Calculation using libs

The scipy and statsmodel examples in the simple example above have keywords that allow supplying an expected distribution. Since the degrees of freedom within the functions are calculated as $k = n- 1 - \textrm{ddof}$, the number of prarameters in the model, $q$ can be supplied as ddof.

In [19]:
scipy.stats.chisquare(df["shirts"], f_exp=df["expected_shirts_normed"], ddof=0)

Power_divergenceResult(statistic=0.018336070060208, pvalue=0.9908738632636134)

In [20]:
statsmodels.stats.gof.chisquare(df["shirts"], f_exp=df["expected_shirts_normed"], ddof=0)

(0.018336070060208, 0.9908738632636134)